In [18]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GITHUB_TOKEN")
secret_value_1 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_2 = user_secrets.get_secret("HuggingFACEHUB_access_token")
secret_value_3 = user_secrets.get_secret("LANGCHAIN_API_KEY")

# ✅ IMPORTANT: Set them in os.environ so other code can access them
os.environ["GITHUB_TOKEN"] = secret_value_0
os.environ["GROQ_API_KEY"] = secret_value_1
os.environ["HuggingFACEHUB_access_token"] = secret_value_2
os.environ["LANGCHAIN_API_KEY"] = secret_value_3
os.environ["LLM_BACKEND"] = "vllm"

# ✅ NEW: Set vLLM model path (must match hirachical-subchat.ipynb)
model_path = "/kaggle/input/qwen-3/transformers/14b-awq/1"
os.environ["VLLM_MODEL_PATH"] = model_path

# Print the tokens (first 4 and last 4 characters for security)
print("="*60)
print("🔐 SECRETS LOADED AND SET IN ENVIRONMENT")
print("="*60)
print(f"✅ GITHUB_TOKEN: {secret_value_0[:4]}...{secret_value_0[-4:]}")
print(f"✅ GROQ_API_KEY: {secret_value_1[:4]}...{secret_value_1[-4:]}")
print(f"✅ HuggingFACEHUB_access_token: {secret_value_2[:4]}...{secret_value_2[-4:]}")
print(f"✅ LANGCHAIN_API_KEY: {secret_value_3[:4]}...{secret_value_3[-4:]}")
print(f"✅ LLM_BACKEND: vllm")
print(f"✅ VLLM_MODEL_PATH: {model_path}")
print("="*60)

🔐 SECRETS LOADED AND SET IN ENVIRONMENT
✅ GITHUB_TOKEN: gith...tWfg
✅ GROQ_API_KEY: gsk_...l6gr
✅ HuggingFACEHUB_access_token: hf_E...GaQC
✅ LANGCHAIN_API_KEY: lsv2...ea2f
✅ LLM_BACKEND: vllm
✅ VLLM_MODEL_PATH: /kaggle/input/qwen-3/transformers/14b-awq/1


In [19]:
# Configure git identity for Kaggle git push
import subprocess

REPO_DIR = "/kaggle/working/Subchat-Trees"

print("="*60)
print("⚙️  CONFIGURING GIT FOR KAGGLE")
print("="*60)

try:
    import os
    os.chdir(REPO_DIR)
    
    # Set git identity
    subprocess.run(["git", "config", "user.name", "moonmehedi"], check=True)
    subprocess.run(["git", "config", "user.email", "the.mehedi.hasan.moon@gmail.com"], check=True)
    
    print("✅ Git identity configured!")
    print(f"   User: moonmehedi")
    print(f"   Email: the.mehedi.hasan.moon@gmail.com")
    
    # Verify current branch
    branch_result = subprocess.run(["git", "branch", "--show-current"], capture_output=True, text=True)
    print(f"\n✅ Current branch: {branch_result.stdout.strip()}")
    
    # Check if GITHUB_TOKEN is available
    if "GITHUB_TOKEN" in os.environ:
        token_preview = os.environ["GITHUB_TOKEN"]
        print(f"\n✅ GITHUB_TOKEN available: {token_preview[:4]}...{token_preview[-4:]}")
    else:
        print("\n⚠️  WARNING: GITHUB_TOKEN not set - git push will fail!")
    
    os.chdir("/kaggle/working")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("="*60)

⚙️  CONFIGURING GIT FOR KAGGLE
✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run

✅ GITHUB_TOKEN available: gith...tWfg


In [ ]:
# ✅ RUN KAGGLE BUFFER TEST RUNNER (with automatic git push after each buffer)
! python /kaggle/working/Subchat-Trees/backend/dataset/kaggle_buffer_test_runner.py

Traceback (most recent call last):
  File "/kaggle/working/Subchat-Trees/backend/dataset/kaggle_buffer_test_runner.py", line 1182, in <module>
    runner = KaggleMetricsTestRunner()
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/kaggle/working/Subchat-Trees/backend/dataset/kaggle_buffer_test_runner.py", line 39, in __init__
    self.classifier = ContextClassifier()
                      ^^^^^^^^^^^^^^^^^^^
  File "/kaggle/working/Subchat-Trees/backend/dataset/context_classifier.py", line 68, in __init__
    raise RuntimeError(
RuntimeError: ❌ vLLM not available for classification!
   Judge/classifier requires vLLM to avoid Groq API quota limits.
   Please ensure VLLMClient.set_model(llm) was called in your notebook.


: 

In [21]:
import subprocess
import os
import time

print("="*60)
print("🛑 SHUTTING DOWN SERVER AND KERNEL")
print("="*60)

# 1. Kill the backend server
try:
    print("\n🔴 Stopping backend server...")
    result = subprocess.run(
        ["pkill", "-f", "uvicorn.*src.main:app"],
        capture_output=True,
        text=True
    )
    time.sleep(2)
    print("✅ Backend server stopped")
except Exception as e:
    print(f"⚠️  Error stopping server: {e}")

# 2. Kill the kernel
print("\n🔴 Terminating kernel...")
print("✅ Kernel will shut down now - this saves GPU quota!")
print("="*60)

# Force exit the kernel
os._exit(0)

🛑 SHUTTING DOWN SERVER AND KERNEL

🔴 Stopping backend server...


: 

: 